In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pylab
import scipy.stats as stat
from scipy.stats import ttest_ind, chi2_contingency
import statsmodels.api as sm
from statsmodels.formula.api import ols
from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
import pickle
import os

In [5]:
os.chdir("C:\\Users\\Nilesh\\Documents\\GitHub\\Credit Card Spam Detection")
df = pd.read_csv("CSD_FS.csv")
df

,trans_date_trans_time,merchant,category,amt,first,last,gender,state,trans_num,unix_time,is_fraud
0,2020-06-21 12:14:25,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,SC,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,0
1,2020-06-21 12:14:33,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,UT,324cc204407e99f51b0d6ca0055005e7,1371816873,0
2,2020-06-21 12:14:53,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,NY,c81755dbbbea9d5c77f094348a7579be,1371816893,0
3,2020-06-21 12:15:15,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,FL,2159175b9efe66dc301f149d3d5abf8c,1371816915,0
4,2020-06-21 12:15:17,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,MI,57ff021bd3f328f8738bb535c302a31b,1371816917,0
...,...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,MO,9b1f753c79894c9f4b71f04581835ada,1388534347,0
555715,2020-12-31 23:59:09,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,TX,2090647dac2c89a1d86c514c427f5b91,1388534349,0
555716,2020-12-31 23:59:15,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,WA,6c5b7c8add471975aa0fec023b2e8408,1388534355,0
555717,2020-12-31 23:59:24,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,ID,14392d723bb7737606b2700ac791b7aa,1388534364,0


In [6]:
df["fullname"] = df['first'].astype(str) +" "+ df["last"].astype(str)

In [8]:
df.drop(['first'], axis=1, inplace=True)
df.drop(['last'], axis=1, inplace=True)

In [9]:
df

,trans_date_trans_time,merchant,category,amt,gender,state,trans_num,unix_time,is_fraud,fullname
0,2020-06-21 12:14:25,fraud_Kirlin and Sons,personal_care,2.86,M,SC,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,0,Jeff Elliott
1,2020-06-21 12:14:33,fraud_Sporer-Keebler,personal_care,29.84,F,UT,324cc204407e99f51b0d6ca0055005e7,1371816873,0,Joanne Williams
2,2020-06-21 12:14:53,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,F,NY,c81755dbbbea9d5c77f094348a7579be,1371816893,0,Ashley Lopez
3,2020-06-21 12:15:15,fraud_Haley Group,misc_pos,60.05,M,FL,2159175b9efe66dc301f149d3d5abf8c,1371816915,0,Brian Williams
4,2020-06-21 12:15:17,fraud_Johnston-Casper,travel,3.19,M,MI,57ff021bd3f328f8738bb535c302a31b,1371816917,0,Nathan Massey
...,...,...,...,...,...,...,...,...,...,...
555714,2020-12-31 23:59:07,fraud_Reilly and Sons,health_fitness,43.77,M,MO,9b1f753c79894c9f4b71f04581835ada,1388534347,0,Michael Olson
555715,2020-12-31 23:59:09,fraud_Hoppe-Parisian,kids_pets,111.84,M,TX,2090647dac2c89a1d86c514c427f5b91,1388534349,0,Jose Vasquez
555716,2020-12-31 23:59:15,fraud_Rau-Robel,kids_pets,86.88,F,WA,6c5b7c8add471975aa0fec023b2e8408,1388534355,0,Ann Lawson
555717,2020-12-31 23:59:24,fraud_Breitenberg LLC,travel,7.99,M,ID,14392d723bb7737606b2700ac791b7aa,1388534364,0,Eric Preston


### Encoding Variables

In [11]:
import pickle
le = LabelEncoder()
df['gender_le'] = le.fit_transform(df['gender'])
pickle.dump(le, open('gender_le.pkl','wb'))
df.drop(['gender'], axis=1, inplace=True)

In [12]:
le = LabelEncoder()
df['merchant_le'] = le.fit_transform(df['merchant'])
df.drop(['merchant'], axis=1, inplace=True)
pickle.dump(le, open('merchant_le.pkl','wb'))

In [13]:
le = LabelEncoder()
df['state_le'] = le.fit_transform(df['state'])
df.drop(['state'], axis=1, inplace=True)
pickle.dump(le, open('state_le.pkl','wb'))

In [14]:
le = LabelEncoder()
df['category_le'] = le.fit_transform(df['category'])
df.drop(['category'], axis=1, inplace=True)
pickle.dump(le, open('category_le.pkl','wb'))

In [15]:
df['trans_num'].value_counts()

2da90c7d74bd46a0caf3777415b3ebd3    1
7b0f0f113f949d3f9f9e3951f720e507    1
19e0a51f95ac46f1c0076b94badde4e2    1
ffb01d7da76c227c9b41f090ce1fe558    1
176f641c4139e069710ae11c997189a2    1
                                   ..
b861f33f7adf1d7b849e0a4e527b76d4    1
739a283be52dfe3caac0dafbc87df525    1
25c80571f38ca752233ab232d592fdc4    1
a953633e35ee9d575cf3c77dd41bf9eb    1
1765bb45b3aa3224b4cdcb6e7a96cee3    1
Name: trans_num, Length: 555719, dtype: int64

In [16]:
df.drop(['trans_num'], axis=1, inplace=True)

In [18]:
le = LabelEncoder()
df['fullname_le'] = le.fit_transform(df['fullname'])
df.drop(['fullname'], axis=1, inplace=True)
pickle.dump(le, open('fullname_le.pkl','wb'))

In [20]:
df.drop(['trans_date_trans_time'], axis=1, inplace=True)

In [21]:
df

,amt,unix_time,is_fraud,gender_le,merchant_le,state_le,category_le,fullname_le
0,2.86,1371816865,0,1,319,39,10,377
1,29.84,1371816873,0,0,591,43,10,428
2,41.28,1371816893,0,0,611,33,5,70
3,60.05,1371816915,0,1,222,8,9,109
4,3.19,1371816917,0,1,292,21,13,678
...,...,...,...,...,...,...,...,...
555714,43.77,1388534347,0,1,507,23,5,644
555715,111.84,1388534349,0,1,264,42,7,452
555716,86.88,1388534355,0,0,496,46,7,59
555717,7.99,1388534364,0,1,75,12,13,285


### Scaling Dataset

In [22]:
df_Y = df['is_fraud']
df_Y

0         0
1         0
2         0
3         0
4         0
         ..
555714    0
555715    0
555716    0
555717    0
555718    0
Name: is_fraud, Length: 555719, dtype: int64

In [23]:
df_X = df.iloc[:,[0,1,3,4,5,6,7]]
df_X

,amt,unix_time,gender_le,merchant_le,state_le,category_le,fullname_le
0,2.86,1371816865,1,319,39,10,377
1,29.84,1371816873,0,591,43,10,428
2,41.28,1371816893,0,611,33,5,70
3,60.05,1371816915,1,222,8,9,109
4,3.19,1371816917,1,292,21,13,678
...,...,...,...,...,...,...,...
555714,43.77,1388534347,1,507,23,5,644
555715,111.84,1388534349,1,264,42,7,452
555716,86.88,1388534355,0,496,46,7,59
555717,7.99,1388534364,1,75,12,13,285


In [24]:
scaler = StandardScaler()
df_X_scaled = pd.DataFrame(scaler.fit_transform(df_X),columns=df_X.columns)

In [25]:
pickle.dump(scaler, open('slr.pkl','wb'))

In [26]:
df_scaled = pd.concat([df_X_scaled, df_Y], axis=1)

In [31]:
df_scaled

,amt,unix_time,gender_le,merchant_le,state_le,category_le,fullname_le,is_fraud
0,-0.424463,-1.703871,1.102494,-0.119663,0.937072,0.961683,-0.327833,0
1,-0.252337,-1.703869,-0.907034,1.231656,1.220424,0.961683,-0.133723,0
2,-0.179353,-1.703865,-0.907034,1.331017,0.512044,-0.316151,-1.496303,0
3,-0.059605,-1.703861,1.102494,-0.601568,-1.258908,0.706116,-1.347865,0
4,-0.422358,-1.703861,1.102494,-0.253802,-0.338013,1.728384,0.817800,0
...,...,...,...,...,...,...,...,...
555714,-0.163467,1.510350,1.102494,0.814337,-0.196337,-0.316151,0.688393,0
555715,0.270803,1.510351,1.102494,-0.392908,1.149586,0.194983,-0.042376,0
555716,0.111564,1.510352,-0.907034,0.759688,1.432939,0.194983,-1.538170,0
555717,-0.391735,1.510354,1.102494,-1.331876,-0.975555,1.728384,-0.677993,0


### Exporting Feature Transformed Data

In [32]:
df_scaled.to_csv('CSD_FT.csv', encoding='utf-8', index=False)